# Crawling ctnd.

In [4]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [6]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
params['query'] = '파이썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
urls = [{'url':_['href'], 'depth':1}
         for _ in dom.select('div.blog.section._blogBase._prs_blg dt > a[href]')]

[Error] https://search.naver.com/search.naver


In [9]:
seen = list() # 방문했는지

while urls: # Queue -> fFIrst in first out -> pop(0), urls이 존재하는 한
    seed = urls.pop(0) # starting url
    seen.append(seed) # 재방문 회피
    if seed['depth'] <2: # 범위 제한, Focused Crawling
        dom = BeautifulSoup(download(seed['url']).text, 'html.parser') # HTTP

        for _ in [_['href'] if _.has_attr('href') else _['src']
            for _ in dom.select('a[href], #mainrame')
             if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',
                         _['href'] if _.has_attr('href') else _['src'])]:
            newUrls = urljoin(seed['url'], _)
            if urlparse(newUrls)[1] in ['blog.naver.com']\
            and newUrls not in [_['url'] for _ in seen] \
            and newUrls not in [_['url'] for _ in urls]: # 도메인 제한
                urls.append({'url':newUrls, 'depth':seed['depth']+1})
        print(len(urls), len(seen))

2 1
1 2
0 3


In [12]:
seen

[{'url': 'https://blog.naver.com/thkimnguy?Redirect=Log&logNo=222016218489',
  'depth': 1},
 {'url': 'https://blog.naver.com/fastcampus?Redirect=Log&logNo=221924690201',
  'depth': 1},
 {'url': 'https://blog.naver.com/alamo_?Redirect=Log&logNo=222028243388',
  'depth': 1}]

In [13]:
urls

[]

In [10]:
import sqlite3

In [115]:
con = sqlite3.connect('page.rank.db')
cur = con.cursor()

In [116]:
cur.executescript('''
    DROP TABLE IF EXISTS host;
    CREATE TABLE host(
        pk       INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc   TEXT NOT NULL
    );
     DROP TABLE IF EXISTS url;
     CREATE TABLE url(
        pk    INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        fk    INTEGER NOT NULL,
        ref   INTEGER NOT NULL,
        path  TEXT NOT NULL,
        crawled TEXT NOT NULL
     );
''')

In [117]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

for _ in dom.select('div.r > a[href]'):
    urlseg = urlparse(_['href']) # url => http://netloc/path?query 이라면
    netloc = '://'.join(urlseg[:2]) # netloc = http://netloc
    path = urlseg[2]+('?'+urlseg[4] if urlseg[4] else '') # path = path?query
    # host table에 netloc이 있는지 확인
    cur.execute('SELECT pk FROM host WHERE netloc =? LIMIT 0,1',
            ['://'.join(urlseg[:2]),])
    # 없다면 host table에 netloc 추가
    if not cur.fetchone():
        cur.execute('INSERT INTO host(netloc) VALUES(?)', ['://'.join(urlseg[:2]),])
        con.commit()
    # url table에 path 추가 (단, host table netloc의 pk를 fk로 가져옴)
    cur.execute('''
    INSERT INTO url(fk, ref, path, crawled)
    VALUES((SELECT pk FROM host WHERE netloc =? LIMIT 0,1), ?,?,'N')
    ''', [netloc, 0, path])
    con.commit()

[Error] https://www.google.com/search


In [118]:
for _ in range(100): # True: or while in range(100)
    cur.execute('''
        SELECT host.pk, url.pk, host.netloc, url.path FROM url
        INNER JOIN host ON host.pk=url.fk
        WHERE url.crawled='N'
        ORDER BY url.pk ASC
        LIMIT 0,1
    ''')
    seed = cur.fetchone()

    if not seed:
        break
        
    cur.execute('''
        UPDATE url SET crawled='Y' WHERE pk=?
    ''', [seed[1],])
    url = ''.join(seed[2:])
    dom = BeautifulSoup(download(url).text, 'html.parser')
    for _ in [_['href'] for _ in dom.select('a[href]')
              if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',
                          _['href'])]:
        urlseg = urlparse(urljoin(url, _))
        netloc = '://'.join(urlseg[:2])
        path = urlseg[2]+('?'+urlseg[4] if urlseg[4] else '')
        cur.execute('''
            SELECT pk FROM host WHERE netloc=?
            ORDER BY pk ASC LIMIT 0,1''', ['://'.join(urlseg[:2]),])
        if not cur.fetchone():
            cur.execute('INSERT INTO host(netloc) VALUES(?)',
                        ['://'.join(urlseg[:2]),])
            con.commit()
        cur.execute('''
            SELECT count(pk) FROM url WHERE fk=? and path=?
        ''',[seed[0], path])
        
        if cur.fetchone()[0] < 1:
            cur.execute('''
            INSERT INTO url(fk, ref, path, crawled)
            VALUES((SELECT pk FROM host WHERE netloc=? ORDER BY pk LIMIT 0, 1), ?, ?, 'N')
            ''', [netloc, seed[0], path])
            con.commit()
    break

In [119]:
cur.execute('SELECT COUNT(pk) FROM host')
cnt = cur.fetchone()[0]

In [120]:
# DB의 reference 가중치 계산하기 - 여기선 data가 별로 없어서 대부분 0, 1로 채워져서 한계
# page rank, domain의 영향력을 볼 수 있음
import numpy as np
A = np.zeros((cnt+1,cnt+1))
v = np.zeros((cnt+1,))
S = np.zeros((cnt+1,))

In [121]:
A.shape, v.shape

((132, 132), (132,))

In [122]:
for j in range(1,cnt+1):
    cur.execute('SELECT fk, count(fk) FROM url WHERE ref=? GROUP BY fk', [j,])
for i in cur:
     A[i[0],j] = i[1]
     S[j] += i[1]

In [123]:
AA = A/S
AA[np.isnan(AA)]=0
# 데이터에 0/0이 많아 nan이 많다.

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [124]:
v[:]=1/cnt

In [128]:
v[1] # 가중치

0.0

In [126]:
for _ in range(100):
    v=AA.dot(v)
    print('Iter:{}'.format(_+1))
    for _ in v.argsort()[::-1][:5]:
        cur.execute('SELECT * FROM host WHERE pk=?', [str(_),])
        print(cur.fetchone(), np.round(v[_],10),
             A[:,_].sum(), A[_].sum())
        print()

Iter:1
(131, 'https://wikimediafoundation.org') 0.0 0.0 0.0

(48, 'https://sr.wikipedia.org') 0.0 0.0 0.0

(34, 'https://af.wikipedia.org') 0.0 0.0 0.0

(35, 'https://ms.wikipedia.org') 0.0 0.0 0.0

(36, 'https://hu.wikipedia.org') 0.0 0.0 0.0

Iter:2
(131, 'https://wikimediafoundation.org') 0.0 0.0 0.0

(48, 'https://sr.wikipedia.org') 0.0 0.0 0.0

(34, 'https://af.wikipedia.org') 0.0 0.0 0.0

(35, 'https://ms.wikipedia.org') 0.0 0.0 0.0

(36, 'https://hu.wikipedia.org') 0.0 0.0 0.0

Iter:3
(131, 'https://wikimediafoundation.org') 0.0 0.0 0.0

(48, 'https://sr.wikipedia.org') 0.0 0.0 0.0

(34, 'https://af.wikipedia.org') 0.0 0.0 0.0

(35, 'https://ms.wikipedia.org') 0.0 0.0 0.0

(36, 'https://hu.wikipedia.org') 0.0 0.0 0.0

Iter:4
(131, 'https://wikimediafoundation.org') 0.0 0.0 0.0

(48, 'https://sr.wikipedia.org') 0.0 0.0 0.0

(34, 'https://af.wikipedia.org') 0.0 0.0 0.0

(35, 'https://ms.wikipedia.org') 0.0 0.0 0.0

(36, 'https://hu.wikipedia.org') 0.0 0.0 0.0

Iter:5
(131, 'https:

## Scrapping

In [ ]:
# https://validator.w3.org/unicorn/?ucn_lang=ko 
# 여기에 www.ppomppu.co.kr 입력해보자.
# for url warnings, errors checking
# error가 많으면 parser가 찾기 힘듦
# 어떤 error가 있고, 어떤 조치를 취해야 하는 지 살펴 볼 것

In [137]:
url = 'https://n.weather.naver.com/'
params = {
    'id':'ppomppu'
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'lxml')

In [98]:
resp.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />\n<meta http-equiv="Content-Script-Type" content="text/javascript" />\n<meta http-equiv="Content-Style-Type" content="text/css" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="author" content="PPOMPPU CO.">\n<meta name="description" content="뽐뿌게시판은 국내 쇼핑몰에서 판매중인 가성비 좋은 상품과 쇼핑정보를 공유하는 공간입니다.">\n<meta name="keywords" content="">\n\n<meta name="og:title" content="뽐뿌">\n<meta name="og:description" content="사람이 좋아 함께하는 곳... 뽐뿌! 쇼핑특가, 휴대폰 정보의 중심지">\n\n<script src="https://www.googleoptimize.com/optimize.js?id=OPT-N6TJ7LN"></script>\n\n<script async=\'async\' src=\'//www.googletagservices.com/tag/js/gpt.js\'></script>\n<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>\n\n<!--\n<meta name="viewport" content="width=device-width, i

In [99]:
dom.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="text/javascript" http-equiv="Content-Script-Type"/>\n  <meta content="text/css" http-equiv="Content-Style-Type"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="PPOMPPU CO." name="author"/>\n  <meta content="뽐뿌게시판은 국내 쇼핑몰에서 판매중인 가성비 좋은 상품과 쇼핑정보를 공유하는 공간입니다." name="description"/>\n  <meta content="" name="keywords"/>\n  <meta content="뽐뿌" name="og:title"/>\n  <meta content="사람이 좋아 함께하는 곳... 뽐뿌! 쇼핑특가, 휴대폰 정보의 중심지" name="og:description"/>\n  <script src="https://www.googleoptimize.com/optimize.js?id=OPT-N6TJ7LN">\n  </script>\n  <script async="async" src="//www.googletagservices.com/tag/js/gpt.js">\n  </script>\n  <script>\n   var googletag = googletag || {};googletag.cmd = googletag.cmd || [];\n  </script>\n  <!--\n<meta name="viewpo

In [138]:
# title pattern
title = re.compile(r'[[]?(\w+)[]]?([^(]+)[(](.+?)[)]')

for _ in dom.select('#revolution_main_table tr[class^=list]'):
    td = _.find_all('td')
    print(td[3].text.strip(), td[-2].text.strip())
    print(title.findall(td[3].text))
#     break

[필독] 제목에는 특수 문자 사용을 절대 금지합니다.- 2010년 2월 24일 0
[]
[G마켓] 전자렌지 용기 400ml 32개(스마일클럽 10,790원/무료)  5 1 - 0
[('G마켓', ' 전자렌지 용기 400ml 32개', '스마일클럽 10,790원/무료')]
[위메프] 슈퍼치즈 왕 돈까스200G (2,850/무료)  6 
[('위메프', ' 슈퍼치즈 왕 돈까스200G ', '2,850/무료')]
[원더폰] 갤럭시 A10e 단순박스개봉 미사용폰 (130,000/무료)  19 
[('원더폰', ' 갤럭시 A10e 단순박스개봉 미사용폰 ', '130,000/무료')]
[위메프]오뚜기참치/고추/야채 150GX10개 차이결제시(10,040원/무료)  4 1 - 0
[('위메프', '오뚜기참치/고추/야채 150GX10개 차이결제시', '10,040원/무료')]
[옥션]LG 통돌이 TR12BL (361,000원 무료)  14 
[('옥션', 'LG 통돌이 TR12BL ', '361,000원 무료')]
[무신사]스탠다드 깅엄 체크 린넨 셔츠(9,900원/무료)  11 1 - 0
[('무신사', '스탠다드 깅엄 체크 린넨 셔츠', '9,900원/무료')]
[지마켓] 육아서 말로 때린 상처가 더 아프다 (11480원/무배)  13 0 - 5
[('지마켓', ' 육아서 말로 때린 상처가 더 아프다 ', '11480원/무배')]
[쿠팡] 스윙칩 60g 12개 (9,900원 무료배송)  10 7 - 0
[('쿠팡', ' 스윙칩 60g 12개 ', '9,900원 무료배송')]
[G마켓] 컬쳐랜드 5만원권 8% (46,000/무료)  24 5 - 1
[('G마켓', ' 컬쳐랜드 5만원권 8% ', '46,000/무료')]
[티몬][끌올] 머지포인트 (89,900원/무료)  16 
[('티몬', '[끌올] 머지포인트 ', '89,900원/무료')]
[비밀의공구] 듀얼브레서 프리다이빙 v3 스노쿨링 마스크( 59,800 / 3,000)  22 
[('비밀의공구', ' 듀얼브레서

In [ ]:
[[]? # 문자 있거나 없거나
 (\w+) # 글자 1개 이상 (1)
 []]? # 문자 있거나 없거나
([^(]+) # 그룹, 문자, (가 아닌 다른 문자 1개 이상 (2)
  [(] # 문자 ( 있고
    (.+?) # 아무 글자나 1개 이상 (3)
    [)] # 문자 있다
    
[(1)] (2) ((3))
[LG] 통돌이 모델명 (가격)

In [139]:
# 무식하게 scrapping
for _ in dom.select('#revolution_main_table tr[class^=list]')[1:]:
    item=dict()
    td = _.find_all('td')
#     print(td[3].text.strip(), td[-2].text.strip())
#     print(title.findall(td[3].text))
    frag1 = title.findall(td[3].text)[0]
    frag2 = td[-2].text.split('-')
    item['mall'] = frag1[0].strip()
    item['title'] = frag1[1].strip()
    item['price'] = frag1[2].strip()
    item['tu'] = frag2[0].strip() if len(frag2) > 1 else 0
    item['td'] = frag2[1].strip() if len(frag2) > 1 else 0
    print(item)
#     break

{'mall': 'G마켓', 'title': '전자렌지 용기 400ml 32개', 'price': '스마일클럽 10,790원/무료', 'tu': '1', 'td': '0'}
{'mall': '위메프', 'title': '슈퍼치즈 왕 돈까스200G', 'price': '2,850/무료', 'tu': 0, 'td': 0}
{'mall': '원더폰', 'title': '갤럭시 A10e 단순박스개봉 미사용폰', 'price': '130,000/무료', 'tu': 0, 'td': 0}
{'mall': '위메프', 'title': '오뚜기참치/고추/야채 150GX10개 차이결제시', 'price': '10,040원/무료', 'tu': '1', 'td': '0'}
{'mall': '옥션', 'title': 'LG 통돌이 TR12BL', 'price': '361,000원 무료', 'tu': 0, 'td': 0}
{'mall': '무신사', 'title': '스탠다드 깅엄 체크 린넨 셔츠', 'price': '9,900원/무료', 'tu': '1', 'td': '0'}
{'mall': '지마켓', 'title': '육아서 말로 때린 상처가 더 아프다', 'price': '11480원/무배', 'tu': '0', 'td': '5'}
{'mall': '쿠팡', 'title': '스윙칩 60g 12개', 'price': '9,900원 무료배송', 'tu': '7', 'td': '0'}
{'mall': 'G마켓', 'title': '컬쳐랜드 5만원권 8%', 'price': '46,000/무료', 'tu': '5', 'td': '1'}
{'mall': '티몬', 'title': '[끌올] 머지포인트', 'price': '89,900원/무료', 'tu': 0, 'td': 0}
{'mall': '비밀의공구', 'title': '듀얼브레서 프리다이빙 v3 스노쿨링 마스크', 'price': '59,800 / 3,000', 'tu': 0, 'td': 0}
{'mall': '동아제약', 't

In [140]:
# Crawling(링크 따라가기) + Scrapping

In [141]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php'
params = {
    'id':'ppomppu',
    'page':1
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

In [143]:
# Focused Crawling - 게시판의 파라미터(page)로 제한
# Crawling
for page in range(1,10):
    params['page']=page
    resp = download(url, params, headers, 'GET')
    dom = BeautifulSoup(resp.text, 'lxml')
    
    # Scraping
    for _ in dom.select('#revolution_main_table tr[class^=list]')[1:]:
        item=dict()
        td = _.find_all('td', recursive =False)
        temp = title.findall(td[3].text)
        frag1 = temp[0] if len(temp) > 0 else td[3].text.strip()
        frag2 = td[-2].text.split('-')

        item['mall'] = frag1[0].strip()
        item['title'] = frag1[1].strip()
        item['price'] = frag1[2].strip()
        item['tu'] = frag2[0].strip() if len(frag2) > 1 else 0
        item['td'] = frag2[1].strip() if len(frag2) > 1 else 0
        print(item)

{'mall': 'G마켓', 'title': '니베아 맨 딥 쉐이빙젤 200ml 1+1. 스마일클럽, 화요카드만', 'price': '9,120/2500', 'tu': 0, 'td': 0}
{'mall': 'G마켓', 'title': '전자렌지 용기 400ml 32개', 'price': '스마일클럽 10,790원/무료', 'tu': '1', 'td': '0'}
{'mall': '위메프', 'title': '슈퍼치즈 왕 돈까스200G', 'price': '2,850/무료', 'tu': 0, 'td': 0}
{'mall': '원더폰', 'title': '갤럭시 A10e 단순박스개봉 미사용폰', 'price': '130,000/무료', 'tu': 0, 'td': 0}
{'mall': '위메프', 'title': '오뚜기참치/고추/야채 150GX10개 차이결제시', 'price': '10,040원/무료', 'tu': '2', 'td': '0'}
{'mall': '옥션', 'title': 'LG 통돌이 TR12BL', 'price': '361,000원 무료', 'tu': 0, 'td': 0}
{'mall': '무신사', 'title': '스탠다드 깅엄 체크 린넨 셔츠', 'price': '9,900원/무료', 'tu': '1', 'td': '0'}
{'mall': '지마켓', 'title': '육아서 말로 때린 상처가 더 아프다', 'price': '11480원/무배', 'tu': '0', 'td': '7'}
{'mall': '쿠팡', 'title': '스윙칩 60g 12개', 'price': '9,900원 무료배송', 'tu': '7', 'td': '0'}
{'mall': 'G마켓', 'title': '컬쳐랜드 5만원권 8%', 'price': '46,000/무료', 'tu': '5', 'td': '1'}
{'mall': '티몬', 'title': '[끌올] 머지포인트', 'price': '89,900원/무료', 'tu': 0, 'td': 0}
{'mall': '비밀

In [144]:
resp.request.url # crawler가 page 9 까지 갔다!

'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=9'

In [149]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

urls = list()
seen = list()

urls.append(url)

# 클릭 가능한 모든 a들 가져오고 싶다.
while urls:
    seed = urls.pop(0)
    seen.append(seed)

    dom = BeautifulSoup(download(seed).text, 'lxml')
    for _ in [_['href']
              for _ in dom.select('''
              tr.list0 a[href], tr.list1 a[href], #page_list a[href]
              ''')
              if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', _['href'])]:
        newUrls = urljoin(seed, _)
        if newUrls not in urls \
        and newUrls not in seen:
            urls.append(newUrls)
    print(len(urls), len(seen))

10 1
10 2
9 3
8 4
7 5
6 6
5 7
4 8
3 9
2 10
11 11
10 12
9 13
8 14
7 15
6 16
5 17
4 18
3 19
2 20
1 21
10 22
9 23
8 24
7 25
6 26
5 27
4 28
3 29
2 30
1 31
10 32
9 33
8 34
7 35
6 36
5 37
4 38
3 39
2 40
1 41
10 42
9 43
8 44
7 45
6 46
5 47
4 48
3 49
2 50
1 51
10 52
9 53
8 54
7 55
6 56
5 57
4 58
3 59
2 60
1 61
10 62
9 63
8 64
7 65
6 66
5 67


KeyboardInterrupt: 

In [153]:
seen # 링크들 모아놓음

['http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=2&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=3&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=4&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=5&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=6&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=7&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=8&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=9&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=10&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=11&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=1&divpage=62',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page=12&divpage=62',
 'http://www.ppomppu.co.kr/

In [195]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php'

params = {
    'id':'freeboard',
    'page':1
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

resp = download(url, params, headers)
dom = BeautifulSoup(resp.text,'lxml')

In [175]:
for _ in dom.select('#page_list a'):
  print(_.text.strip()) # page

2
3
4
5
6
7
8
9
10
다음


In [196]:
resp = download(url, params, headers)
dom = BeautifulSoup(resp.text,'lxml') 
# 글목록
for _ in dom.select('.list0 > td:nth-of-type(3) > a, .list1 > td:nth-of-type(3) > a'):
    if _['href'].startswith('v'):
        print(urljoin(url, _['href']))
        resp = download(urljoin(url, _['href']))
        dom = BeautifulSoup(resp.text, 'lxml')
        if dom.select_one('.board-contents') != None:
            print(dom.select_one('.board-contents').name)

resp = download(url, params, headers)
dom = BeautifulSoup(resp.text,'lxml') 
# 게시판 목록
for _ in dom.select('#page_list a'):
    if _['href'].startswith('/'):
        print(urljoin(url, _['href']))
        resp = download(urljoin(url, _['href']))
        dom = BeautifulSoup(resp.text, 'lxml')
        if dom.select_one('.board-contents') != None:
            print(dom.select_one('.board-contents').name)

http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080352
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080350
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080348
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080346
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080344
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080342
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080340
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080338
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080336
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080334
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080332
td
http://www.ppomppu.co.kr/zboard/

In [205]:
# 합치면
urls = list()
seen = list()

urls.append({'url':url, 'depth':1})

while urls:
    seed = urls.pop(0)
    seen.append(seed)
    
    if seed['depth'] > 3:
        break
        
    resp = download(seed['url'])
    dom = BeautifulSoup(resp.text, 'lxml')
    
    # 한 줄로
    for _ in dom.select('.list0 > td:nth-of-type(3) a, .list1 > td:nth-of-type(3) a, #page_list a'):
        if _['href'][0] in ['v', '/']:
            newurl = urljoin(url, _['href'])
            if newurl not in [_['url'] for _ in urls] and\
               newurl not in [_['url'] for _ in seen]:
                   urls.append({'url':newurl,
                                'depth':seed['depth']+1})
            # 여기까지가 크롤링
            # 여기서부터가 스크래핑
            # 제목과 내용을 스크래핑할 것이다
    if dom.select_one('.board-contents') != None:
            print(
                {'head':dom.select_one('.view_title2').text.strip(),
                 'body':dom.select_one('.board-contents').text.strip(),
                 'comments': [_.text.strip()
                              for _ in dom.select('.comment_wrapper .han')]})
print()

In [182]:
dom.select('.board-contents')

[]